# AUTOCORRECTOR : With min edit distance

In [1]:
import numpy as np
from fuzzy import Soundex
import re
import ast

## THE KEYBOARD PROXIMITY

In [2]:
KEY = ast.literal_eval(open("./pa_files/qwerty_graph.txt").read())

def find_key(key): 
    return KEY[key]

def neighboor_keys(key): 
    return [KEY[key][k] for k in KEY[key].keys()]

In [3]:
def del_cost():
    return 1

def ins_cost():
    return 1

def sub_cost(a, b):
    if a == b :
        return 0
    else:
        if a in neighboor_keys(b):
            return 1
        else:
            return 2
        
def trans_cost(w01, w02, w11, w12):
    if w01 == w02 and w11 == w12:
        return 2
    else :
        return 4 

## EDIT DISTANCE ALGORITHM 

In [4]:
def back_trace(edit_matrix, source, target):
    
    dist_matrix = np.empty((len(source)+1, len(target)+1), dtype=object)
    
    dist_matrix[0,:] = 'D'
    dist_matrix[:,0] = 'L'
    dist_matrix[0,0] = '0'
    
    for i in range(1, len(source) + 1):
        for j in range(1, len(target) + 1):
            
            dele = edit_matrix[i-1, j] + del_cost()
            ins = edit_matrix[i, j-1] + ins_cost()
            sub = edit_matrix[i-1, j-1] + sub_cost(source[i-1], target[j-1])

            min_cost = min(dele, ins, sub)

            edit_ops = ""
            if min_cost == dele:
                edit_ops += 'D'
            if min_cost == ins:
                edit_ops += 'L'
            if min_cost == sub:
                edit_ops += 'G'

            dist_matrix[i, j] = edit_ops
         
    return dist_matrix
                

In [5]:

def min_edit_distance(source, target, do_print_chart=False, do_back_trace=False):

    edit_matrix = np.zeros((len(source)+1, len(target)+1))
    edit_matrix[:, 0] = [i for i in range(len(source) + 1)]
    edit_matrix[0,:] = [j for j in range(len(target) + 1)]
    
    for i in range(1, len(source)+1):
        for j in range(1, len(target)+1):
            edit_matrix[i,j] = min(edit_matrix[i-1,j] + del_cost(), 
                               edit_matrix[i,j-1] + ins_cost() , 
                               edit_matrix[i-1,j-1] + sub_cost(source[i-1], target[j-1]))

    if(do_print_chart):
        print(edit_matrix)
        
    if(do_back_trace):
        print(back_trace(edit_matrix,source,target))
        
    return edit_matrix[-1,-1]

In [6]:
min_edit_distance("tail", "tall", do_print_chart=True, do_back_trace=True)

[[0. 1. 2. 3. 4.]
 [1. 0. 1. 2. 3.]
 [2. 1. 0. 1. 2.]
 [3. 2. 1. 1. 2.]
 [4. 3. 2. 1. 1.]]
[['0' 'D' 'D' 'D' 'D']
 ['L' 'G' 'L' 'L' 'L']
 ['L' 'D' 'G' 'L' 'L']
 ['L' 'D' 'D' 'G' 'LG']
 ['L' 'D' 'D' 'G' 'G']]


1.0

## AutoCorrector V.1

In [7]:
def words(text): return re.findall(r'\w+', text.lower())

In [8]:
WORDS = words(open('./pa_files/big.txt').read())

In [9]:
def candidates(word): 
    
    set_of_words= set(WORDS)

    words_with_med = [(w,min_edit_distance(w, word)) for w in set_of_words]

    words_with_med = sorted(words_with_med, key=lambda x: x[1])

    return words_with_med


In [10]:

def candidates_correction(word , k=1):
    if(len(candidates(word)) > k):
        return candidates(word)[0:k]
    else :
        return candidates(word)


In [11]:
candidates_correction("goergous", k=10)

[('gorgeous', 2.0),
 ('nervous', 3.0),
 ('onerous', 3.0),
 ('forgo', 4.0),
 ('generous', 4.0),
 ('gros', 4.0),
 ('grievous', 4.0),
 ('porous', 4.0),
 ('gorge', 4.0),
 ('governors', 4.0)]

## Optimiser Autocorrecteur

In [12]:
def lengths_close(word1, word2, threshold=2):
    return abs(len(word1) - len(word2)) <= threshold 

In [13]:
def candidates_v2(word): 

    soundex = Soundex(4)
    
    tab = [w for w in [w for w in [w for w in set(WORDS) 
                                   if w[0] == word[0]]
                                    if soundex(word)==soundex(w)] 
                                     if lengths_close(word, w)]
    
    words_with_med = sorted([(w,min_edit_distance(w, word)) for w in tab] , key=lambda x: x[1])
    return words_with_med



In [14]:
def candidates_correction(word , k=1):
    
    if(len(candidates_v2(word)) > k):
        return candidates_v2(word)[0:k]
    else :
        return candidates_v2(word)

In [15]:
candidates_correction("goergos", k=10)

[('gorge', 3.0),
 ('gorgeous', 3.0),
 ('gorges', 3.0),
 ('gowers', 3.0),
 ('georgia', 4.0),
 ('grows', 4.0),
 ('george', 4.0),
 ('gross', 4.0),
 ('georges', 4.0),
 ('growers', 4.0)]

In [18]:
candidates_correction("worm", k=10)

[('worm', 0.0),
 ('worn', 1.0),
 ('warm', 2.0),
 ('warn', 3.0),
 ('warne', 4.0),
 ('warman', 4.0),
 ('weren', 4.0),
 ('warren', 5.0)]

#### made by :
- *Moudni Houda* : houda.moudni@etu.uae.ac.ma
- *Mountassir Chadi* : chadi.mountassir@etu.uae.ac.ma